In [1]:
def get_data(host, user, password, db):

    conn = pymysql.connect(host=host, #localhost
                            user=user, #user
                            password=password,
                            db = db, #youtube
                            charset='utf8mb4')
    cursor = conn.cursor()

    # ==== select example ====
    sql = "select caption from caption"
    cursor.execute(sql)

    # 데이타 Fetch
    captions = cursor.fetchall()
    captions = list(captions)
    captions = [i[0] for i in captions]
    cursor.close()
    return captions

In [2]:
import re

# '[Music, Applause, ...]' <- 전처리 and remove tuple

def preprocessing(captions):
    clean_captions = []

    for caption in captions:
        caption = re.sub("[^a-zA-Z]", " ", str(caption)).strip()
        words = caption.lower().split()
        clean_caption = ' '.join(words)
        clean_captions.append(clean_caption)
    return clean_captions


In [3]:
from sklearn.metrics.pairwise import cosine_similarity
from operator import itemgetter

# extract top 5

def extract_top5(captions, clean_captions, tfidf_matrix):
    checked_similarity = []

    for i in range(len(clean_captions)):
        # 10th sentence
        if cosine_similarity(tfidf_matrix[10], tfidf_matrix[i]) != 0:

            checked_similarity.append([captions[i], cosine_similarity(tfidf_matrix[10], tfidf_matrix[i])])
            
    checked_similarity = sorted(checked_similarity, key=itemgetter(1), reverse=True)
    top_5 = checked_similarity[1:6]
    return top_5

['I have never had a feeling as pure a',
 'proud as completing a mission all you',
 "everything we've done in the last 17",
 'years trying to make a difference only',
 'never took it done',
 "you've been found five times for your",
 "country and you can't even afford to",
 'send your kids to college I got a job',
 "for you I'm retired fish I need a pilot",
 'I got thinking baby now this can change',
 "you in that baby's life forever we",
 'finally get to use our skills for our',
 "own benefit all right let's do it",
 "we're gonna get gabrielle Martina Luria",
 'head of one of the biggest drug cartels',
 "I've got estimates he's got over 75",
 "million dollars in cash if we're not",
 'gone forever after you make your move or',
 "you're dead",
 'you cannot go back to your normal life',
 'after tonight you guys need to own the',
 "fact what we're about to do is criminal",
 'everyone to the office right now Jesus',
 'Christ',
 ' regards their back we got to go all',
 'right we can use take

In [4]:
# from nltk.corpus import stopwords
# from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from PreProcessing import preprocessing
from Get_data import get_data
from Extract_top import extract_top5

input_sentence = input('Input sentence: ')

captions = get_data('localhost', 'root', '111111', db='youtube')
captions.append(input_sentence)

clean_captions = preprocessing(captions)

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(clean_captions)

# extract top 5

top_5 = extract_top5(captions, clean_captions, tfidf_matrix)

for i in range(len(top_5)):
    print("\n" + str(i+1) + ". " + top_5[i][0] + " | similarity: " + str(top_5[i][1][0][0]))


Input sentence: i love you

1. l | similarity: 0.4213819993593776

2. w | similarity: 0.3983809458743322

3. m | similarity: 0.38001115919729545

4. y | similarity: 0.3693997968522127

5. y | similarity: 0.3693997968522127


In [7]:
top_5

[['l', array([[0.421382]])],
 ['w', array([[0.39838095]])],
 ['m', array([[0.38001116]])],
 ['y', array([[0.3693998]])],
 ['y', array([[0.3693998]])]]